In [8]:
import os
import time
import sys
import numpy as np
from scipy import sparse, io

debug = 0
old_settings = np.seterr(all='ignore')
## own functions
sys.path.append(".")

from deeptools import parserCommon, bamHandler, utilities
from deeptools.mapReduce import mapReduce
from deeptools.utilities import smartLabels

In [9]:
import argparse
from scFilterStats import getFiltered_worker
from scFilterStats import parseArguments
#from deeptools.estimateReadFiltering import getFiltered_worker
#from deeptools.estimateReadFiltering import parseArguments

In [16]:
import pandas as pd

In [10]:
bamfiles = [ '/hpc/hub_oudenaarden/vbhardwaj/2019_scHiC/04_ESC_NPC/04_workflow_outputs/fESC_CTCF-K4me3_26-02-2020/dedup_bam/HVG-ChIC-fESC-CTCF-i1-26-02-20.bam',
             '/hpc/hub_oudenaarden/vbhardwaj/2019_scHiC/04_ESC_NPC/04_workflow_outputs/fESC_CTCF-K4me3_26-02-2020/dedup_bam/HVG-ChIC-fESC-CTCF-i2-26-02-20.bam',
             '/hpc/hub_oudenaarden/vbhardwaj/2019_scHiC/04_ESC_NPC/04_workflow_outputs/fESC_CTCF-K4me3_26-02-2020/dedup_bam/HVG-ChIC-fESC-CTCF-i3-26-02-20.bam',
             '/hpc/hub_oudenaarden/vbhardwaj/2019_scHiC/04_ESC_NPC/04_workflow_outputs/fESC_CTCF-K4me3_26-02-2020/dedup_bam/HVG-ChIC-fESC-CTCF-i4-26-02-20.bam'
           ]

In [11]:
args = parseArguments()
args = args.parse_args('-b testdata/test.bam -o testdata/outstats.txt --barcodes bc'.split())
## make barcode list
args.barcodes = ['GCGATGCA', 'TGTACACT', 'GACAGCGC']
args.bamfiles = bamfiles#['testdata/test.bam', 'testdata/test2.bam']
args.motifFilter = ['A', 'TA']
args.genome2bit =  "testdata/mouse_ensembl_97.2bit"
args.sampleLabels = ['i1', 'i2', 'i3', 'i4']

In [12]:
    if args.outFile is None:
        of = sys.stdout
    else:
        of = open(args.outFile, "w")

    bhs = [bamHandler.openBam(x, returnStats=True, nThreads=args.numberOfProcessors) for x in args.bamfiles]

    ## need to convert these stats to single-cells
    #mapped = [x[1] for x in bhs]
    #unmappedList = [x[2] for x in bhs]
    bhs = [x[0] for x in bhs]

    chrom_sizes = list(zip(bhs[0].references, bhs[0].lengths))
    for x in bhs:
        x.close()

In [13]:
    # Get the remaining metrics
    res = mapReduce([args],
                    getFiltered_worker,
                    chrom_sizes,
                    genomeChunkLength=args.binSize + args.distanceBetweenBins,
                    blackListFileName=args.blackListFileName,
                    numberOfProcessors=args.numberOfProcessors,
                    verbose=args.verbose)

In [14]:
arr2 = np.asarray(res).sum(axis = 0)

array([[21549, 12782,     0,     0,     0,     0,     0,  3575,     0,
        11025,     0],
       [15666,  9263,     0,     0,     0,     0,     0,  2342,     0,
         8148,     0],
       [ 9957,  5622,     0,     0,     0,     0,     0,  1576,     0,
         4762,     0],
       [  278,   231,     0,     0,     0,     0,     0,    72,     0,
          210,     0],
       [16262, 10179,     0,     0,     0,     0,     0,  2600,     0,
         9036,     0],
       [   24,    18,     0,     0,     0,     0,     0,     3,     0,
           17,     0],
       [51465, 28364,     0,     0,     0,     0,     0,  9228,     0,
        23283,     0],
       [   65,    52,     0,     0,     0,     0,     0,    15,     0,
           47,     0],
       [   21,    15,     0,     0,     0,     0,     0,     4,     0,
           14,     0],
       [55400, 51709,     0,     0,     0,     0,     0,  9715,     0,
        50935,     0],
       [  194,   175,     0,     0,     0,     0,     0,    

In [27]:
rowLabels = ["{}_{}".format(a, b) for a in args.sampleLabels for b in args.barcodes ]

In [34]:
colLabels = ["Total","Filtered","Low_MAPQ",
             "Missing_Flags","Excluded_Flags","Internal_Duplicates",
             "Marked_Duplicates","Singletons","Wrong_strand",
             "Wrong_motif", "unwanted_GC_content"]

In [35]:
final_df = pd.DataFrame(data = np.concatenate(arr2),
                  index = rowLabels,
                  columns = colLabels)

In [ ]:
final_df.iloc[:,1:] = final_df.iloc[:,1:].div(final_df.Total, axis=0)*100